# Identify the World's Oldest Businesses

<center><img src="MKn_Staffelter_Hof.jpeg" alt="Picture of old business"</center>
<!--Image Credit: Martin Kraft https://commons.wikimedia.org/wiki/File:MKn_Staffelter_Hof.jpg -->

Staffelter Hof Winery is Germany's oldest business, established in 862 under the Carolingian dynasty. It has continued to serve customers through dramatic changes in Europe, such as the Holy Roman Empire, the Ottoman Empire, and both world wars. What characteristics enable a business to stand the test of time?

To help answer this question, BusinessFinancing.co.uk researched the oldest company still in business in **almost** every country and compiled the results into several CSV files. This dataset has been cleaned.

Having useful information in different files is a common problem. While it's better to keep different types of data separate for data storage, you'll want all the data in one place for analysis. You'll use joining and data manipulation to work with this data and better understand the world's oldest businesses.

## The Data
`businesses` and `new_businesses`
|Column|Description|
|------|-----------|
|`business`|Name of the business (varchar)|
|`year_founded`|Year the business was founded (int)|
|`category_code`|Code for the business category (varchar)|
|`country_code`|ISO 3166-1 three-letter country code (char)|
---
`countries`
|Column|Description|
|------|-----------|
|`country_code`|ISO 3166-1 three-letter country code (varchar)|
|`country`|Name of the country (varchar)|
|`continent`|Name of the continent the country exists in (varchar)|
---
`categories`
|Column|Description|
|------|-----------|
|`category_code`|Code for the business category (varchar)|
|`category`|Description of the business category (varchar)|

In [ ]:
%%sql

-- Deterine the old business on each continent by year

WITH combine_tables AS (
	SELECT *
	FROM public.businesses
	UNION
	SELECT *
	FROM public.new_businesses
)

SELECT
	subquery.continent,
	subquery.business,
	subquery.country,
	subquery.year_founded
FROM (
	SELECT
		ct.business,
		ct.year_founded,
		c.country,
		c.continent,
		ROW_NUMBER() OVER (PARTITION BY c.continent ORDER BY ct.year_founded ASC) AS rn
	FROM combine_tables AS ct
	LEFT JOIN public.countries AS c
	ON ct.country_code = c.country_code
) subquery
WHERE rn = 1

,continent,business,country,year_founded
0,Africa,Mauritius Post,Mauritius,1772
1,Asia,Kongō Gumi,Japan,578
2,Europe,St. Peter Stifts Kulinarium,Austria,803
3,North America,La Casa de Moneda de México,Mexico,1534
4,Oceania,Australia Post,Australia,1809
5,South America,Casa Nacional de Moneda,Peru,1565


In [ ]:
%%sql

-- Determine how many countries in each continent are missing records on new and old businesses

WITH combine_tables AS (
	SELECT *
	FROM public.businesses
	UNION ALL
	SELECT *
	FROM public.new_businesses
)

SELECT
	c.continent,
	COUNT(*) AS countries_without_businesses
FROM public.countries AS c
	LEFT JOIN combine_tables AS ct
	ON ct.country_code = c.country_code
WHERE ct.business IS NULL
GROUP BY c.continent

,continent,countries_without_businesses
0,Africa,3
1,Asia,7
2,Europe,2
3,North America,5
4,Oceania,10
5,South America,3


In [ ]:
%%sql

-- Identify oldest business category per continent based on founding year

SELECT
	c.continent,
	cat.category,
	MIN (ct.year_founded) AS year_founded
FROM public.businesses AS ct
INNER JOIN public.countries AS c
	ON ct.country_code = c.country_code
INNER JOIN public.categories AS cat
	ON cat.category_code = ct.category_code
GROUP BY c.continent, cat.category
ORDER BY c.continent, year_founded DESC

,continent,category,year_founded
0,Africa,Energy,1968
1,Africa,Mining,1962
2,Africa,Agriculture,1947
3,Africa,Media,1943
4,Africa,"Distillers, Vintners, & Breweries",1933
5,Africa,Banking & Finance,1892
6,Africa,Food & Beverages,1878
7,Africa,Aviation & Transport,1854
8,Africa,Manufacturing & Production,1820
9,Africa,Postal Service,1772
